## イネの病気を判別するためのモデル(Model which clasifies the disease of rice plant)
The model classfies the diseases of rice plant. Diseases includes "rice blast" and "stripes".
When this program run, artificial intelligence classfies the disease in Flask that is right framework made by python.
It depends on images that AI classfies diseases. So, if students would make the model, they must collect clear images about the diseases.

In [0]:
from keras.applications.inception_v3  import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, AveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, CSVLogger, LearningRateScheduler, ReduceLROnPlateau
from keras.optimizers import SGD
from keras import optimizers
from keras.regularizers import l2
import matplotlib.image as mpimg
from scipy.misc import imresize
from keras.callbacks import EarlyStopping 
import numpy as np
import keras.backend as K
import math

In [63]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [64]:
## Google Drive内のデータを確認
!ls ./gdrive/'My Drive'/"Colab Notebooks"

class.ipynb			riceplantordog.ipynb  ver
model				train (1)	      verificate.ipynb
model_classfies_diseases.ipynb	Untitled0.ipynb
rice-plant-man.ipynb		validation


In [0]:
## 多値分類のコードを追加していく

### 元モデルをinceptionV3にセット
base_model = InceptionV3(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(256, activation="relu")(x)
predictions = Dense(2, activation="sigmoid")(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

model.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])

In [81]:
test_datagen = ImageDataGenerator(rescale=1.0 / 255)


### 学習用のデータをかさ増し
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    horizontal_flip = True,
    fill_mode = "nearest",
    zoom_range = 0.3,
    width_shift_range = 0.3,
    height_shift_range=0.3,
    rotation_range=90)
train_generator = train_datagen.flow_from_directory(
    './gdrive/My Drive/Colab Notebooks/train (1)',
    target_size = (256, 256), ## デフォルト
    batch_size = 32,
    class_mode = "categorical") ## 多値分類なのでcategoricalに設定

Found 100 images belonging to 2 classes.


In [82]:
## 学習用データの水増し
validation_generator = test_datagen.flow_from_directory(
    './gdrive/My Drive/Colab Notebooks/validation/',
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical')

Found 42 images belonging to 2 classes.


In [83]:
print(train_generator.class_indices)

{'imoti': 0, 'shima': 1}


In [84]:
checkpoint = ModelCheckpoint("./gdrive/My Drive/Colab Notebooks/saver/dis_{epoch:02d}.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')

model.fit_generator(train_generator, 
                   steps_per_epoch=None, 
                   epochs=10, 
                   verbose=1, 
                   callbacks=None, 
                   validation_data=None, 
                   validation_steps=None, 
                   class_weight=None, 
                   max_queue_size=10, 
                   workers=1, 
                   use_multiprocessing=False, 
                   shuffle=True, 
                   initial_epoch=0)

Epoch 1/10
4/4 [==============================] - 19s 5s/step - loss: 0.7042 - acc: 0.5272
Epoch 2/10
4/4 [==============================] - 3s 765ms/step - loss: 0.7382 - acc: 0.4975
Epoch 3/10
4/4 [==============================] - 3s 738ms/step - loss: 0.7256 - acc: 0.4893
Epoch 4/10
4/4 [==============================] - 3s 797ms/step - loss: 0.7217 - acc: 0.4480
Epoch 5/10
4/4 [==============================] - 3s 751ms/step - loss: 0.6909 - acc: 0.5165
Epoch 6/10
4/4 [==============================] - 3s 742ms/step - loss: 0.7107 - acc: 0.4918
Epoch 7/10
4/4 [==============================] - 3s 743ms/step - loss: 0.6701 - acc: 0.5603
Epoch 8/10
4/4 [==============================] - 3s 745ms/step - loss: 0.6928 - acc: 0.5413
Epoch 9/10
4/4 [==============================] - 3s 731ms/step - loss: 0.6875 - acc: 0.5355
Epoch 10/10
4/4 [==============================] - 3s 737ms/step - loss: 0.6696 - acc: 0.5495
